用 Bagpipes 拟合目录数据
=====================

通常我们希望拟合一个目录中多个对象的观测（例如 Guo et al. (2013) 的 CANDELS GOODS South 目录，见上例）。

一种方法是将前面示例中的拟合命令放入 for 循环，但 Bagpipes 提供了一个通过 `fit_catalogue` 类进行目录拟合的接口（参见文档），使操作更方便。此外还提供了若干 MPI 并行化选项。

设置
------

我们将使用示例 3 的设置来演示目录拟合的工作原理。首先复制 `load_data` 函数并生成拟合指令字典。

In [ ]:
import numpy as np 
import bagpipes as pipes

from astropy.io import fits

def load_goodss(ID):
    """ 从 Guo et al. (2013) 目录加载 CANDELS GOODS South 的光度数据。 """

    # 从目录中加载相关列。
    cat = np.loadtxt("data/hlsp_candels_hst_wfc3_goodss-tot-multiband_f160w_v1-1photom_cat.txt",
                     usecols=(10, 13, 16, 19, 25, 28, 31, 34, 37, 40, 43, 46, 49, 52, 55,
                              11, 14, 17, 20, 26, 29, 32, 35, 38, 41, 44, 47, 50, 53, 56))
    
    # 找到我们需要对象的正确行。
    row = int(ID) - 1

    # 从目录中提取目标对象。
    fluxes = cat[row, :15]
    fluxerrs = cat[row, 15:]

    # 将它们转换为二维数组。
    photometry = np.c_[fluxes, fluxerrs]

    # 将与缺失通量相关的误差放大。
    for i in range(len(photometry)):
        if (photometry[i, 0] == 0.) or (photometry[i, 1] <= 0):
            photometry[i,:] = [0., 9.9*10**99.]
            
    # 对于前十个通道强制最大信噪比为20，IRAC通道最大为10。
    for i in range(len(photometry)):
        if i < 10:
            max_snr = 20.
            
        else:
            max_snr = 10.
        
        if photometry[i, 0]/photometry[i, 1] > max_snr:
            photometry[i, 1] = photometry[i, 0]/max_snr

    return photometry

goodss_filt_list = np.loadtxt("filters/goodss_filt_list.txt", dtype="str")


exp = {}                                  
exp["age"] = (0.1, 15.)
exp["tau"] = (0.3, 10.)
exp["massformed"] = (1., 15.)
exp["metallicity"] = (0., 2.5)

dust = {}
dust["type"] = "Calzetti"
dust["Av"] = (0., 2.)

fit_instructions = {}
fit_instructions["redshift"] = (0., 10.)
fit_instructions["exponential"] = exp   
fit_instructions["dust"] = dust


基本目录拟合
----------------

在最基本的情况下，你只需要一组 ID。将该列表与 `fit_instructions` 和 `load_data` 一同传递给 `fit_catalogue` 即可。拟合通过调用 `fit` 方法开始，使用方式与普通的 `fit` 类相同。下面我们开始拟合 Guo et al. 目录中的前三个对象。

In [ ]:
IDs = np.arange(1, 4)

fit_cat = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="guo_cat")

fit_cat.fit(verbose=False)



Bagpipes: fitting object 1


Completed in 253.3 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.846      0.999      1.205
exponential:age                1.308      1.481      1.847
exponential:massformed        10.626     10.689     10.756
exponential:metallicity        0.678      1.535      2.127
exponential:tau                0.318      0.366      0.479
redshift                       0.474      0.504      0.527




Bagpipes: 1 out of 3 objects completed.

Bagpipes: fitting object 2


Completed in 207.0 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.148      0.319      0.506
exponential:age                2.974      3.267      3.467
exponential:massformed        10.263     10.366     10.441
exponential:metallicity        2.256      2.407      2.475
exponential:tau                1.180      1.656      3.618
redshift                       1.762      1.850      1.918


Bagpipes: 2 out of 3 objects completed.

Bagpipes: fitting object 3


Completed in 210.9 seconds.

Parameter                          Posterior percentiles
                                16th       50th       84th
----------------------------------------------------------
dust:Av                        0.519      0.616      0.685
exponential:age                2.156 

## 输出目录

摘要目录将自动保存在 `pipes/cats/run_name.fits`。

更复杂的选项
----------------

有一些其他选项可能会很有用。例如，如果你有要拟合对象的光谱红移列表，你可能希望将每次拟合的红移固定为不同的值。你可以通过将红移值数组作为 `redshifts` 参数传入来实现这一点。

In [ ]:
redshifts = np.ones(IDs.shape)

cat_fit = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=goodss_filt_list, run="guo_cat_redshift_1",
                              redshifts=redshifts)


如果你希望在输入红移附近的一个小范围内改变红移，可以向 `redshift_sigma` 参数传入一个浮点数。这将使每个对象的红移以传入的 `redshifts` 值为中心，使用指定的标准差进行高斯先验拟合。允许的最大偏离为给定 `redshift_sigma` 的三倍。

## 改变 `filt_list`

最后，如果你有一组具有不同光度的对象，并希望用相同模型拟合它们，可以将滤镜列表组成的列表传递给 `fit_catalogue` 的 `cat_filt_list` 参数。如果这样做，需要将 `vary_filt_list` 参数设为 `True`，代码会将 `cat_filt_list` 的第一项作为第一个对象的滤镜列表，依此类推。下面演示使用相同的滤镜列表为每个对象设置：


In [ ]:
list_of_filt_lists = [goodss_filt_list] * 10

cat_fit = pipes.fit_catalogue(IDs, fit_instructions, load_goodss, spectrum_exists=False,
                              cat_filt_list=list_of_filt_lists, run="guo_cat_vary_filt_list",
                              redshifts=redshifts, redshift_sigma=0.05, vary_filt_list=True)


## MPI 并行化

`fit_catalogue` 支持与 `fit` 相同的 MPI 并行化（见示例 3）。此外，还可以请求 `fit_catalogue` 将不同对象分配给可用的不同核心，同时并行拟合多个对象。这对于在大量核心上运行或对大型光度目录拟合简单模型时更快，但单个拟合将耗时更久。通过将 `mpi_serial` 参数设为 `True` 可以实现此功能。要以这种方式运行 Bagpipes，目前需要一个经过轻微修改的 `pymultinest` 版本。